In [1]:
import pandas as pd 
from sklearn.linear_model import LogisticRegression
import json
from sklearn import metrics
import warnings
warnings.filterwarnings('ignore')
#from label_decoders import *

In [2]:
config = json.load(open('settings.json'))
train = pd.read_csv(r'D:\marlabs\Datasets\prudential\train.csv')
test = pd.read_csv(r'D:\marlabs\Datasets\prudential\test.csv')

In [3]:
all_data = train.append(test)


In [5]:
all_data.head()

,BMI,Employment_Info_1,Employment_Info_2,Employment_Info_3,Employment_Info_4,Employment_Info_5,Employment_Info_6,Family_Hist_1,Family_Hist_2,Family_Hist_3,...,Medical_Keyword_9,Product_Info_1,Product_Info_2,Product_Info_3,Product_Info_4,Product_Info_5,Product_Info_6,Product_Info_7,Response,Wt
0,0.323008,0.028,12,1,0.0,3,NaN,2,NaN,0.598039,...,0,1,D3,10,0.076923,2,1,1,8.0,0.148536
1,0.272288,0.000,1,3,0.0,2,0.0018,2,0.188406,NaN,...,0,1,A1,26,0.076923,2,3,1,4.0,0.131799
2,0.428780,0.030,9,1,0.0,2,0.0300,3,0.304348,NaN,...,0,1,E1,26,0.076923,2,3,1,8.0,0.288703
3,0.352438,0.042,9,1,0.0,3,0.2000,3,0.420290,NaN,...,0,1,D4,10,0.487179,2,3,1,8.0,0.205021
4,0.424046,0.027,9,1,0.0,2,0.0500,2,0.463768,NaN,...,0,1,D2,26,0.230769,2,3,1,8.0,0.234310


In [7]:
all_data['Product_Info_2_char'] = all_data.Product_Info_2.str[0]
all_data['Product_Info_2_num'] = all_data.Product_Info_2.str[1]
all_data['Product_Info_2'] = pd.factorize(all_data['Product_Info_2'])[0]
all_data['Product_Info_2_char'] = pd.factorize(all_data['Product_Info_2_char'])[0]
all_data['Product_Info_2_num'] = pd.factorize(all_data['Product_Info_2_num'])[0]
all_data['BMI_Age'] = all_data['BMI'] * all_data['Ins_Age']
med_keyword_columns = all_data.columns[all_data.columns.str.startswith('Medical_Keyword_')]
all_data['Med_Keywords_Count'] = all_data[med_keyword_columns].sum(axis=1)
all_data.apply(lambda x: sum(x.isnull()),1)
all_data['countna'] = all_data.apply(lambda x: sum(x.isnull()),1)
all_data.fillna(-1, inplace=True)
all_data['Response'] = all_data['Response'].astype(int)

In [9]:
all_data.shape

(79146, 133)

In [6]:
#all_data.apply(lambda x: sum(x.isnull()),1)

0         6
1         5
2         7
3         7
4         8
5         6
6         7
7         6
8         6
9         7
10        7
11        7
12        7
13        8
14        6
15        6
16        7
17        7
18        6
19        7
20        6
21        6
22        7
23        6
24        6
25        8
26        8
27        9
28        6
29        5
         ..
19735     8
19736     6
19737     6
19738     8
19739     7
19740     9
19741    10
19742     6
19743     8
19744     7
19745     7
19746     6
19747     6
19748    10
19749     7
19750     6
19751     7
19752     7
19753     7
19754     8
19755     7
19756     8
19757     8
19758     7
19759     9
19760     7
19761     7
19762     6
19763     9
19764     4
Length: 79146, dtype: int64

In [11]:
all_data.Product_Info_2.unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18], dtype=int64)

In [12]:
all_data.head()

,BMI,Employment_Info_1,Employment_Info_2,Employment_Info_3,Employment_Info_4,Employment_Info_5,Employment_Info_6,Family_Hist_1,Family_Hist_2,Family_Hist_3,...,Product_Info_5,Product_Info_6,Product_Info_7,Response,Wt,Product_Info_2_char,Product_Info_2_num,BMI_Age,Med_Keywords_Count,countna
0,0.323008,0.028,12,1,0.0,3,-1.0000,2,-1.000000,0.598039,...,2,1,1,8,0.148536,0,0,0.207304,0,6
1,0.272288,0.000,1,3,0.0,2,0.0018,2,0.188406,-1.000000,...,2,3,1,4,0.131799,1,1,0.016256,0,5
2,0.428780,0.030,9,1,0.0,2,0.0300,3,0.304348,-1.000000,...,2,3,1,8,0.288703,2,1,0.012799,0,7
3,0.352438,0.042,9,1,0.0,3,0.2000,3,0.420290,-1.000000,...,2,3,1,8,0.205021,0,2,0.057863,1,7
4,0.424046,0.027,9,1,0.0,2,0.0500,2,0.463768,-1.000000,...,2,3,1,8,0.234310,0,3,0.177213,0,8


In [13]:
train_ohd = all_data[all_data['Response']>0].copy()
test_ohd = all_data[all_data['Response']<1].copy()


In [15]:
train_ohd.columns

Index(['BMI', 'Employment_Info_1', 'Employment_Info_2', 'Employment_Info_3',
       'Employment_Info_4', 'Employment_Info_5', 'Employment_Info_6',
       'Family_Hist_1', 'Family_Hist_2', 'Family_Hist_3',
       ...
       'Product_Info_5', 'Product_Info_6', 'Product_Info_7', 'Response', 'Wt',
       'Product_Info_2_char', 'Product_Info_2_num', 'BMI_Age',
       'Med_Keywords_Count', 'countna'],
      dtype='object', length=133)

In [16]:
features=train_ohd.columns.tolist()
features = [x.replace('=','_') for x in features]
features = [x.replace('_','i') for x in features]
train_ohd.columns = features
features_t=test_ohd.columns.tolist()
features_t = [x.replace('=','i') for x in features_t]
features_t = [x.replace('_','i') for x in features_t]
test_ohd.columns = features_t

In [18]:
features.remove("Id")
features.remove("Response")

In [19]:
train_ohd.head()

,BMI,EmploymentiInfoi1,EmploymentiInfoi2,EmploymentiInfoi3,EmploymentiInfoi4,EmploymentiInfoi5,EmploymentiInfoi6,FamilyiHisti1,FamilyiHisti2,FamilyiHisti3,...,ProductiInfoi5,ProductiInfoi6,ProductiInfoi7,Response,Wt,ProductiInfoi2ichar,ProductiInfoi2inum,BMIiAge,MediKeywordsiCount,countna
0,0.323008,0.028,12,1,0.0,3,-1.0000,2,-1.000000,0.598039,...,2,1,1,8,0.148536,0,0,0.207304,0,6
1,0.272288,0.000,1,3,0.0,2,0.0018,2,0.188406,-1.000000,...,2,3,1,4,0.131799,1,1,0.016256,0,5
2,0.428780,0.030,9,1,0.0,2,0.0300,3,0.304348,-1.000000,...,2,3,1,8,0.288703,2,1,0.012799,0,7
3,0.352438,0.042,9,1,0.0,3,0.2000,3,0.420290,-1.000000,...,2,3,1,8,0.205021,0,2,0.057863,1,7
4,0.424046,0.027,9,1,0.0,2,0.0500,2,0.463768,-1.000000,...,2,3,1,8,0.234310,0,3,0.177213,0,8


In [22]:
train_ohd['lr1'] = [0]*train_ohd.shape[0]
train_ohd['lr2'] = [0]*train_ohd.shape[0]
train_ohd['lr3'] = [0]*train_ohd.shape[0]
train_ohd['lr4'] = [0]*train_ohd.shape[0]
train_ohd['lr5'] = [0]*train_ohd.shape[0]
train_ohd['lr6'] = [0]*train_ohd.shape[0]
train_ohd['lr7'] = [0]*train_ohd.shape[0]
train_ohd['lr8'] = [0]*train_ohd.shape[0]
train_ohd['lr9'] = [0]*train_ohd.shape[0]
train_ohd['lr10'] = [0]*train_ohd.shape[0]
train_ohd['lr11'] = [0]*train_ohd.shape[0]
train_ohd['lr12'] = [0]*train_ohd.shape[0]
train_ohd['lr13'] = [0]*train_ohd.shape[0]

In [23]:
train_ohd.head()

,BMI,EmploymentiInfoi1,EmploymentiInfoi2,EmploymentiInfoi3,EmploymentiInfoi4,EmploymentiInfoi5,EmploymentiInfoi6,FamilyiHisti1,FamilyiHisti2,FamilyiHisti3,...,lr4,lr5,lr6,lr7,lr8,lr9,lr10,lr11,lr12,lr13
0,0.323008,0.028,12,1,0.0,3,-1.0000,2,-1.000000,0.598039,...,0,0,0,0,0,0,0,0,0,0
1,0.272288,0.000,1,3,0.0,2,0.0018,2,0.188406,-1.000000,...,0,0,0,0,0,0,0,0,0,0
2,0.428780,0.030,9,1,0.0,2,0.0300,3,0.304348,-1.000000,...,0,0,0,0,0,0,0,0,0,0
3,0.352438,0.042,9,1,0.0,3,0.2000,3,0.420290,-1.000000,...,0,0,0,0,0,0,0,0,0,0
4,0.424046,0.027,9,1,0.0,2,0.0500,2,0.463768,-1.000000,...,0,0,0,0,0,0,0,0,0,0


In [53]:
s=10
#range(0,59381//10),
[(range(s//10,s//10*2), list(filter(lambda x: x not in range(s//10,s//10*2), range(0,s)))),
 range(s//10*2,s//10*3), list(filter(lambda x: x not in range(s//10*2,s//10*3), range(0,s)))]

[(range(1, 2), [0, 2, 3, 4, 5, 6, 7, 8, 9]),
 range(2, 3),
 [0, 1, 3, 4, 5, 6, 7, 8, 9]]

In [51]:
range(1,3)

range(1, 3)

In [55]:
l = train_ohd.shape[0]
ind_list = [(range(0,l//10), list(filter(lambda x: x not in range(0,l//10), range(0,l)))), 
            (range(l//10,l//10*2), list(filter(lambda x: x not in range(l//10,l//10*2), range(0,l)))),
            (range(l//10*2,l//10*3), list(filter(lambda x: x not in range(l//10*2,l//10*3), range(0,l)))),
            (range(l//10*3,l//10*4), list(filter(lambda x: x not in range(l//10*3,l//10*4), range(0,l)))),
            (range(l//10*4,l//10*5), list(filter(lambda x: x not in range(l//10*4,l//10*5), range(0,l)))),
            (range(l//10*5,l//10*6), list(filter(lambda x: x not in range(l//10*5,l//10*6), range(0,l)))),
            (range(l//10*6,l//10*7), list(filter(lambda x: x not in range(l//10*6,l//10*7), range(0,l)))),
            (range(l//10*7,l//10*8), list(filter(lambda x: x not in range(l//10*7,l//10*8), range(0,l)))),
            (range(l//10*8,l//10*9), list(filter(lambda x: x not in range(l//10*8,l//10*9), range(0,l)))),
            (range(l//10*9,l), list(filter(lambda x: x not in range(l//10*9,l), range(0,l))))]

In [56]:
def labels_decoder1(lb):
    lb = [1 if x == 1 else 0 for x in lb]    
    return lb


def labels_decoder2(lb):
    lb = [1 if x == 2 else 0 for x in lb]    
    return lb


def labels_decoder3(lb):
    lb = [1 if x == 3 else 0 for x in lb]    
    return lb


def labels_decoder4(lb):
    lb = [1 if x == 4 else 0 for x in lb]    
    return lb


def labels_decoder5(lb):
    lb = [1 if x == 5 else 0 for x in lb]    
    return lb


def labels_decoder6(lb):
    lb = [1 if x == 6 else 0 for x in lb]    
    return lb


def labels_decoder7(lb):
    lb = [1 if x == 7 else 0 for x in lb]    
    return lb 


def labels_decoder8(lb):
    lb = [1 if x == 8 else 0 for x in lb]    
    return lb


def labels_decoder9(lb):
    lb = [1 if x < 3 else 0 for x in lb]    
    return lb


def labels_decoder10(lb):
    lb = [1 if x < 4 else 0 for x in lb]    
    return lb


def labels_decoder11(lb):
    lb = [1 if x < 5 else 0 for x in lb]    
    return lb


def labels_decoder12(lb):
    lb = [1 if x < 6 else 0 for x in lb]    
    return lb


def labels_decoder13(lb):
    lb = [1 if x < 7 else 0 for x in lb]    
    return lb

In [57]:
ld = [labels_decoder1,labels_decoder2,labels_decoder3,labels_decoder4,labels_decoder5,labels_decoder6,labels_decoder7,
      labels_decoder8,labels_decoder9,labels_decoder10,labels_decoder11,labels_decoder12,labels_decoder13]



In [86]:
i = 0
for l in ld:
    i = i + 1    
    
    for j in range(10):
        
        X_1, X_2 = ind_list[j][1], ind_list[j][0]
        y_1, y_2 = train_ohd.iloc[X_1]['Response'], train_ohd.iloc[X_2]['Response']
        
        lr = LogisticRegression(random_state=1)
        lr.fit(train_ohd[features].iloc[X_1],l(y_1))
        train_ohd['lr%s' % (i)].iloc[X_2] = lr.predict_proba(train_ohd[features].iloc[X_2]).T[1]

In [89]:
train_ohd.to_csv(config['train_lr'],index=0)


In [90]:
y = train_ohd['Response']

i = 0
for l in ld:
    i = i + 1    

###1
    lr = LogisticRegression(random_state=1)
    lr.fit(train_ohd[features],l(y)), i
    test_ohd['lr%s' % (i)] = lr.predict_proba(test_ohd[features]).T[1]

test_ohd.to_csv(config['test_lr'],index=0)